In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget -nc -O 2024swunivchallenge.zip https://huggingface.co/datasets/sw24/sw24/resolve/main/data.zip?download=true && unzip -q -n 2024swunivchallenge.zip
!pip3 -q install natsort ftfy

File ‘2024swunivchallenge.zip’ already there; not retrieving.


In [ ]:
!pip install git+https://github.com/facebookresearch/metaclip.git

  Cloning https://github.com/facebookresearch/metaclip.git to /tmp/pip-req-build-2mzm6bhm
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/metaclip.git /tmp/pip-req-build-2mzm6bhm
  Resolved https://github.com/facebookresearch/metaclip.git to commit 2bbbfad7385b6f214af3d54f16add72dfd094771
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-37x2jofc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-37x2jofc
  Resolved https://github.com/huggingface/transformers to commit 0a7af19f4dc868bafc82f35eb7e8d13bac87a594
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done




---



In [ ]:
device = 'cuda:0'
root = './'
dataset_name = 'Scene'

In [ ]:
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
from torchvision.transforms import RandomAdjustSharpness

In [ ]:
processor_23 = T.Compose(
    [
         T.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
         T.CenterCrop(size=(224, 224)),
         T.Lambda(lambda img: img.convert('RGB')),
         RandomAdjustSharpness(2.3, p=1),
         T.ToTensor(),
         T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
     ]
)

In [ ]:
processor_20 = T.Compose(
    [
         T.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
         T.CenterCrop(size=(224, 224)),
         T.Lambda(lambda img: img.convert('RGB')),
         RandomAdjustSharpness(2, p=1),
         T.ToTensor(),
         T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
     ]
)

In [ ]:
import os, json

In [ ]:
with open(os.path.join(root, 'classes.json'), 'r') as j:
     class_names = json.loads(j.read())

In [ ]:
prompts = []
for class_name in class_names:
    prompts.append(f"{class_name}")
    prompts.append(f"a photo of a {class_name}")
    prompts.append(f"a image of a {class_name}")
    prompts.append(f"art of the {class_name}")
    prompts.append(f"a blurry photo of a {class_name}")
    prompts.append(f"a blurry image of a {class_name}")
    prompts.append(f"a blurred photo of a {class_name}")
    prompts.append(f"a blurred image of a {class_name}")
    prompts.extend([
        f'a bad photo of a {class_name}.',
        f'a photo of many {class_name}.',
        f'a photo of the hard to see {class_name}.',
        f'a low resolution photo of the {class_name}.',
        f'a bad photo of the {class_name}.',
        f'a cropped photo of the {class_name}.',
        f'a photo of a hard to see {class_name}.',
        f'a bright photo of a {class_name}.',
        f'a photo of a clean {class_name}.',
        f'a photo of a dirty {class_name}.',
        f'a dark photo of the {class_name}.',
        f'a photo of my {class_name}.',
        f'a photo of the cool {class_name}.',
        f'a bright photo of the {class_name}.',
        f'a cropped photo of a {class_name}.',
        f'a photo of the dirty {class_name}.',
        f'a jpeg corrupted photo of a {class_name}.',
        f'a blurry photo of the {class_name}.',
        f'a photo of the {class_name}.',
        f'a good photo of the {class_name}.',
        f'a rendering of the {class_name}.',
        f'a {class_name} in a video game.',
        f'a photo of one {class_name}.',
        f'a close-up photo of the {class_name}.',
        f'the {class_name} in a video game.',
        f'a sketch of a {class_name}.',
        f'a low resolution photo of a {class_name}.',
        f'a photo of the clean {class_name}.',
        f'a photo of a large {class_name}.',
        f'a photo of a nice {class_name}.',
        f'a photo of a weird {class_name}.',
        f'a sketch of the {class_name}.',
        f'a jpeg corrupted photo of the {class_name}.',
        f'a good photo of a {class_name}.',
        f'a photo of the nice {class_name}.',
        f'a photo of the small {class_name}.',
        f'a photo of the weird {class_name}.',
        f'a drawing of the {class_name}.',
        f'a photo of the large {class_name}.',
        f'a dark photo of a {class_name}.',
        f'a photo of a small {class_name}.'
    ])

    if class_name == "Buildings":
        prompts.extend([
            "A picture of an urban area with buildings",
            "An architectural structure in the city",
            "The Windows"
        ])
    elif class_name == "Forests":
        prompts.extend([
            "A picture of a dense forest with trees",
            "A scenic view of a forest landscape",
            "A picture of the Trees"
        ])
    elif class_name == "Glacier":
        prompts.extend([
            "A picture of an ice",
            "A scenic view of a snowy glacier",
            "A scenic view of some snow in the mountains"
        ])
    elif class_name == "Mountains":
        prompts.extend([
            "A picture of a mountain range",
            "A scenic view of the mountains",
            "A stunning panorama of rugged mountain cliffs"
        ])
    elif class_name == "Sea":
        prompts.extend([
            "A picture of water",
            "A picture of the ocean",
            "A scenic view of the sea and waves"
        ])
    elif class_name == "Street":
        prompts.extend([
            "A picture of a road",
            "A picture of a busy street in the city",
            "An urban street with buildings and cars"
        ])

<hr>

In [ ]:
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import open_clip, torch
from open_clip import tokenizer
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
model = open_clip.create_model('ViT-bigG-14-quickgelu', pretrained='metaclip_2_5b').to(device)
### pretrained='/content/drive/MyDrive/G14_fullcc2.5b.pt'

100%|██████████████████████████████████████| 30.5G/30.5G [02:42<00:00, 188MiB/s]


In [ ]:
ds_meta = ImageFolder(os.path.join(root, dataset_name), transform=processor_23)
ds_meta.samples = natsorted(ds_meta.samples)
dl_meta = DataLoader(ds_meta, shuffle=False, batch_size=32, num_workers=2)

In [ ]:
meta_probs_list = []

with torch.no_grad(), torch.cuda.amp.autocast():
    text = tokenizer.tokenize(prompts).to(device)
    text_features = model.encode_text(text)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for x, y in tqdm(dl_meta):
        x = x.to(device)
        image_features = model.encode_image(x)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        zero_shot_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        meta_probs_list += zero_shot_probs

100%|██████████| 254/254 [01:01<00:00,  4.11it/s]




---



In [ ]:
del model, text, text_features, x, image_features, zero_shot_probs
torch.cuda.empty_cache()



---



In [ ]:
from transformers import AutoModel, CLIPTokenizer

In [ ]:
tokenizer = CLIPTokenizer.from_pretrained('BAAI/EVA-CLIP-18B')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
os.environ['TRANSFORMERS_OFFLINE']='1'
model = AutoModel.from_pretrained(
         '/content/drive/MyDrive/EVA18B',
         torch_dtype=torch.float16,
         trust_remote_code=True).to('cuda').eval()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# model = AutoModel.from_pretrained(
#     'BAAI/EVA-CLIP-18B',
#     torch_dtype=torch.float16,
#     trust_remote_code=True).to('cuda').eval()

In [ ]:
ds_eva = ImageFolder(os.path.join(root, dataset_name), transform=processor_20)
ds_eva.samples = natsorted(ds_eva.samples)
dl_eva = DataLoader(ds_eva, shuffle=False, batch_size=32, num_workers=2)

with open(os.path.join(root, 'classes.json'), 'r') as j:
     class_names = json.loads(j.read())

In [ ]:
eva_probs_list = []

with torch.no_grad(), torch.cuda.amp.autocast():
    text = tokenizer(prompts, return_tensors='pt', padding=True).input_ids.to('cuda')
    text_features = model.encode_text(text)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for x, y in tqdm(dl_eva):
        x = x.to(device)
        image_features = model.encode_image(x)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        zero_shot_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        eva_probs_list += zero_shot_probs

100%|██████████| 254/254 [06:30<00:00,  1.54s/it]




---

# 1. 054+046

In [ ]:
ensembled_probs_list = [meta_probs * 0.54 + eva_probs * 0.46 for meta_probs, eva_probs in zip(meta_probs_list, eva_probs_list)]

In [ ]:
label_list = [ensembled_probs.reshape(len(class_names), -1).mean(dim=-1).max(dim=-1)[1].tolist() for ensembled_probs in ensembled_probs_list]

In [ ]:
submission = dict({'id_idx':list(range(8100)), 'label': label_list})

In [ ]:
pd.DataFrame(submission).to_csv(os.path.join(root, 'submission1_054_046.csv'), index=False)



---

# 2. 054+046 (> 0.002)

In [ ]:
ensembled_probs_list = [torch.where(ensembled_probs > 0.002, ensembled_probs, 0) for ensembled_probs in ensembled_probs_list] ### ensembled_probs_list ###

In [ ]:
label_list = [ensembled_probs.reshape(len(class_names), -1).mean(dim=-1).max(dim=-1)[1].tolist() for ensembled_probs in ensembled_probs_list]

In [ ]:
submission = dict({'id_idx':list(range(8100)), 'label': label_list})

In [ ]:
pd.DataFrame(submission).to_csv(os.path.join(root, 'submission2_054_046_drp0002.csv'), index=False)



---

# 3. 055+045

In [ ]:
ensembled_probs_list = [meta_probs * 0.55 + eva_probs * 0.45 for meta_probs, eva_probs in zip(meta_probs_list, eva_probs_list)]

In [ ]:
label_list = [ensembled_probs.reshape(len(class_names), -1).mean(dim=-1).max(dim=-1)[1].tolist() for ensembled_probs in ensembled_probs_list]

In [ ]:
submission = dict({'id_idx':list(range(8100)), 'label': label_list})

In [ ]:
pd.DataFrame(submission).to_csv(os.path.join(root, 'submission3_055_045.csv'), index=False)



---

# 4. 055+045 (> 0.002)

In [ ]:
ensembled_probs_list = [torch.where(ensembled_probs > 0.002, ensembled_probs, 0) for ensembled_probs in ensembled_probs_list] ### ensembled_probs_list ###

In [ ]:
label_list = [ensembled_probs.reshape(len(class_names), -1).mean(dim=-1).max(dim=-1)[1].tolist() for ensembled_probs in ensembled_probs_list]

In [ ]:
submission = dict({'id_idx':list(range(8100)), 'label': label_list})

In [ ]:
pd.DataFrame(submission).to_csv(os.path.join(root, 'submission4_055_045_drp0002.csv'), index=False)



---

# 5. 050+050

In [ ]:
ensembled_probs_list = [meta_probs * 0.5 + eva_probs * 0.5 for meta_probs, eva_probs in zip(meta_probs_list, eva_probs_list)]

In [ ]:
label_list = [ensembled_probs.reshape(len(class_names), -1).mean(dim=-1).max(dim=-1)[1].tolist() for ensembled_probs in ensembled_probs_list]

In [ ]:
submission = dict({'id_idx':list(range(8100)), 'label': label_list})

In [ ]:
pd.DataFrame(submission).to_csv(os.path.join(root, 'submission5_050_050.csv'), index=False)



---

# 6. 050+050 (> 0.002)

In [ ]:
ensembled_probs_list = [torch.where(ensembled_probs > 0.002, ensembled_probs, 0) for ensembled_probs in ensembled_probs_list] ### ensembled_probs_list ###

In [ ]:
label_list = [ensembled_probs.reshape(len(class_names), -1).mean(dim=-1).max(dim=-1)[1].tolist() for ensembled_probs in ensembled_probs_list]

In [ ]:
submission = dict({'id_idx':list(range(8100)), 'label': label_list})

In [ ]:
pd.DataFrame(submission).to_csv(os.path.join(root, 'submission6_050_050_drp0002.csv'), index=False)